In [ ]:
import sys
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
import scipy.ndimage.interpolation  as interplt
from PIL import Image
import h5py

ROOT_DIR = os.path.abspath("../src")
sys.path.append(ROOT_DIR)

import data_augmentation as aug
import utils
import constants

In [ ]:
load_file = os.path.join(constants.DATA_DIR, 'train_fpha.h5')
f = h5py.File(load_file, 'r')
print('keys=', list(f.keys()))
img0 = f['img0'][...]
img1 = f['img1'][...]
img2 = f['img2'][...]
uvd_norm_gt = f['uvd_norm_gt'][...]
uvd_gt = f['uvd_gt'][...]
xyz_gt = f['xyz_gt'][...]
hand_center_uvd = f['hand_center_uvd'][...]
file_name = f['file_name'][...]
f.close()

In [ ]:
img0_ori = img0.copy()
img1_ori = img1.copy()
img2_ori = img2.copy()

img0 = np.expand_dims(img0, -1)
img1 = np.expand_dims(img1, -1)
img2 = np.expand_dims(img2, -1)

img0, img1, img2, uvd_norm_gt = aug.augment_data_3d_rot_scale(img0, img1, img2, uvd_norm_gt)

idx=10000
cur_frame = file_name[idx]
depth = Image.open(cur_frame)
depth = np.asarray(depth, dtype='uint16')

imgcopy=depth.copy()
min = imgcopy.min()
max = imgcopy.max()
#scale to 0 - 255
imgcopy = (imgcopy - min) / (max - min) * 255. 
imgcopy = imgcopy.astype('uint8')
imgcopy = cv2.cvtColor(imgcopy, cv2.COLOR_GRAY2BGR)

print(cur_frame)

fig, ax = plt.subplots()
ax.imshow(imgcopy)
utils.visualize_joints_2d(ax, uvd_gt[idx][constants.REORDER], joint_idxs=False)

In [ ]:
img0 = np.squeeze(img0, -1)
img1 = np.squeeze(img1, -1)
img2 = np.squeeze(img2, -1)

fig, ax = plt.subplots(1,3,figsize=(18, 16))
ax[0].imshow(img0[idx])
ax[0].set_title('img0')
ax[1].imshow(img1[idx])
ax[1].set_title('img1')
ax[2].imshow(img2[idx])
ax[2].set_title('img2')

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(18, 16))
ax[0].imshow(img0_ori[idx])
ax[0].set_title('img0')
ax[1].imshow(img1_ori[idx])
ax[1].set_title('img1')
ax[2].imshow(img2_ori[idx])
ax[2].set_title('img2')

In [ ]:
uvd_norm_gt = np.reshape(uvd_norm_gt, (-1, 21, 3))

new_xyz_gt, new_uvd_gt = utils.normuvd2xyzuvd_batch_depth(uvd_norm_gt, hand_center_uvd)

fig, ax = plt.subplots()
ax.imshow(imgcopy)
utils.visualize_joints_2d(ax, new_uvd_gt[idx][constants.REORDER], joint_idxs=False)